In [ ]:
import numpy as np
import pandas as pd
import os
import requests
from PIL import Image
from io import BytesIO
import tensorflow as tf
from tensorflow import keras
from keras.utils import Sequence
from imblearn.over_sampling import SMOTE
from collections import Counter

Image.LOAD_TRUNCATED_IMAGES = True
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Silenciar mensajes de INFO y WARNING

print("TensorFlow version:", tf.__version__)
print("Is GPU available:", tf.config.list_physical_devices('GPU'))

In [ ]:
class AdaptiveDataGenerator(Sequence):
    def __init__(self, df, batch_size, target_size, augment=False, **kwargs):
        super().__init__(**kwargs)  # Llamar al constructor de la clase base
        self.df = df
        self.batch_size = batch_size
        self.target_size = target_size
        self.augment = augment
        self.indices = np.arange(len(self.df))
        self.on_epoch_end()

        # Capas de aumento de datos de Keras
        self.data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip("horizontal_and_vertical"),
            tf.keras.layers.RandomRotation(0.2),
            tf.keras.layers.RandomZoom(0.2),
            tf.keras.layers.RandomContrast(0.2),
        ])

        # Calcular el desbalanceo de clases
        self.class_counts = Counter(np.argmax(self.df[['direccion', 'fachada', 'envio', 'etiqueta']].values, axis=1))
        self.max_class_count = max(self.class_counts.values())

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indices]
        images = []
        labels = []
        for _, row in batch_df.iterrows():
            image = self.load_image(row)
            label = row[['direccion', 'fachada', 'envio', 'etiqueta']].values.astype(np.float32)
            images.append(image)
            labels.append(label)
        images = np.array(images)
        labels = np.array(labels)

        # Aplicar data augmentation adaptativa
        if self.augment:
            augmented_images = []
            augmented_labels = []
            for i in range(len(images)):
                class_idx = np.argmax(labels[i])
                if self.class_counts[class_idx] < self.max_class_count:
                    # Aplicar data augmentation a las clases minoritarias
                    augmented_image = self.data_augmentation(images[i], training=True)
                    augmented_images.append(augmented_image)
                    augmented_labels.append(labels[i])
            if augmented_images:
                images = np.concatenate([images, np.array(augmented_images)], axis=0)
                labels = np.concatenate([labels, np.array(augmented_labels)], axis=0)

        return images, labels

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

    def load_image(self, row):
        if pd.notna(row['filename']):
            img_path = os.path.join(
                "/home/tulua/Documents/Projects/ClassifierImage/repo_dataset/", row['filename']
            )
            # Cargar imagen desde archivo local usando Pillow
            image = Image.open(img_path)
        elif pd.notna(row['urlAbsoluta']):
            # Descargar imagen desde URL
            response = requests.get(row['urlAbsoluta'])
            if response.status_code == 200:
                image = Image.open(BytesIO(response.content))
            else:
                raise ValueError(f"No se pudo descargar la imagen desde {row['urlAbsoluta']}")
        else:
            raise ValueError("No se pudo cargar la imagen.")

        # Convertir a RGB (en caso de que la imagen esté en otro formato, como RGBA)
        if image.mode != 'RGB':
            image = image.convert('RGB')
            
        # Redimensionar la imagen
        image = image.resize((224, 224))  # Redimensionar a 224x224 para MobileNetV3
        # Convertir a un array de numpy y normalizar
        image = np.array(image) / 255.0  # Normalizar
        return image

    def get_all_labels(self):
        """
        Obtiene todas las etiquetas después de aplicar SMOTE y data augmentation.
        """
        all_labels = []
        for i in range(len(self)):
            _, labels = self[i]
            all_labels.extend(labels)
        return np.array(all_labels)

In [ ]:
def load_image(row):
    if pd.notna(row['filename']):
        # Cargar imagen desde archivo local
        img_path = os.path.join(
                "/home/tulua/Documents/Projects/ClassifierImage/repo_dataset/", row['filename']
            )
        # Cargar imagen desde archivo local usando Pillow
        image = Image.open(img_path)
    elif pd.notna(row['urlAbsoluta']):
        # Descargar imagen desde URL
        response = requests.get(row['urlAbsoluta'])
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
        else:
            raise ValueError(f"No se pudo descargar la imagen desde {row['urlAbsoluta']}")
    else:
        raise ValueError(f"No se pudo cargar la imagen. {row['filename']}")
    
        # Convertir a RGB (en caso de que la imagen esté en otro formato, como RGBA)
        if image.mode != 'RGB':
            image = image.convert('RGB')
            
    # Redimensionar la imagen
    image = image.resize((224, 224))  # Redimensionar a 224x224 para MobileNetV3
    # Convertir a un array de numpy y normalizar
    image = np.array(image) / 255.0  # Normalizar
    return image

In [ ]:
def calcular_porcentaje_clases(df, clases):
    """
    Calcula el porcentaje de cada clase en el DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame con las etiquetas.
        clases (list): Lista de nombres de las columnas de las clases.
    
    Returns:
        dict: Diccionario con el porcentaje de cada clase.
    """
    # Calcular el número de muestras por clase
    conteo_clases = df[clases].sum().to_dict()
    
    # Calcular el porcentaje de cada clase
    total_muestras = len(df)
    porcentaje_clases = {clase: (conteo / total_muestras) * 100 for clase, conteo in conteo_clases.items()}
    
    return porcentaje_clases

In [ ]:
# Calcular el porcentaje de cada clase
def calcular_porcentaje_clases_augmented(labels, clases):
    """
    Calcula el porcentaje de cada clase en las etiquetas.
    
    Args:
        labels (np.array): Etiquetas después de SMOTE y data augmentation.
        clases (list): Lista de nombres de las columnas de las clases.
    
    Returns:
        dict: Diccionario con el porcentaje de cada clase.
    """
    conteo_clases = np.sum(labels, axis=0)
    total_muestras = len(labels)
    porcentaje_clases = {clase: (conteo / total_muestras) * 100 for clase, conteo in zip(clases, conteo_clases)}
    return porcentaje_clases

In [ ]:
from sklearn.model_selection import train_test_split

# Cargar el archivo CSV
df = pd.read_csv('./mobilnet-multi-label.csv')

# Seleccionar solo el 40% de los datos
df_sample = df.sample(frac=0.4, random_state=42)

# Dividir el subconjunto en 50% para entrenamiento y 50% para validación
train_df, val_df = train_test_split(df_sample, test_size=0.5, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {len(train_df)}")
print(f"Tamaño del conjunto de validación: {len(val_df)}")

# Calcular el porcentaje de cada clase en el conjunto de entrenamiento
clases = ['direccion', 'fachada', 'envio', 'etiqueta']
porcentaje_clases = calcular_porcentaje_clases(train_df, clases)

# Mostrar el porcentaje de cada clase
print("Porcentaje de cada clase en el conjunto de entrenamiento:")
for clase, porcentaje in porcentaje_clases.items():
    print(f"{clase}: {porcentaje:.2f}%")

# Crear un generador de datos para aplicar SMOTE
def smote_data_generator(df, batch_size, target_size):
    while True:
        # Cargar un lote de imágenes y etiquetas
        batch_df = df.sample(n=batch_size)
        images = np.array([load_image(row) for _, row in batch_df.iterrows()])
        labels = batch_df[['direccion', 'fachada', 'envio', 'etiqueta']].values
        
        # Aplanar las imágenes para SMOTE
        images_flattened = images.reshape(images.shape[0], -1)
        
        # Aplicar SMOTE
        smote = SMOTE(random_state=42)
        images_resampled, labels_resampled = smote.fit_resample(images_flattened, labels)
        
        # Volver a la forma original de las imágenes
        images_resampled = images_resampled.reshape(-1, *target_size, 3)
        
        yield images_resampled, labels_resampled

# Crear generadores de datos
batch_size = 8
target_size = (224, 224)
train_generator = AdaptiveDataGenerator(
    train_df, 
    batch_size, 
    target_size, 
    augment=True, 
    workers=4,  # Pasar aquí los parámetros de optimización
    use_multiprocessing=True,
    max_queue_size=10)
val_generator = AdaptiveDataGenerator(
    val_df, 
    batch_size, 
    target_size, 
    augment=False, 
    workers=4,  # Pasar aquí los parámetros de optimización
    use_multiprocessing=True,
    max_queue_size=10)

# Obtener todas las etiquetas después de SMOTE y data augmentation
all_labels = train_generator.get_all_labels()

# Calcular el porcentaje de cada clase
clases = ['direccion', 'fachada', 'envio', 'etiqueta']
porcentaje_clases = calcular_porcentaje_clases_augmented(all_labels, clases)

# Mostrar el porcentaje de cada clase
print("Porcentaje de cada clase después de SMOTE y data augmentation:")
for clase, porcentaje in porcentaje_clases.items():
    print(f"{clase}: {porcentaje:.2f}%")

In [ ]:
from keras.applications import MobileNetV3Large
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Cargar MobileNetV3 con pesos preentrenados en ImageNet
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir capas personalizadas para la clasificación multi-label
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='sigmoid')(x)  # 4 etiquetas: direccion, fachada, envio, etiqueta

# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=10,
    verbose=1
)

In [ ]:
# Evaluar el modelo en el conjunto de validación
loss, accuracy = model.evaluate(val_generator, verbose=1)
print(f'Loss: {loss}, Accuracy: {accuracy}')

In [ ]:
import matplotlib.pyplot as plt

# Graficar la pérdida y la precisión durante el entrenamiento
def plot_metrics(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Pérdida durante el entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Precisión durante el entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Precisión')
    plt.legend()

    plt.show()

plot_metrics(history)

In [ ]:
# Guardar el modelo entrenado
model.save("mobilenetv3_classifier.v.4.keras")